### Threads (of the same process) run in a shared memory space.

### The threads may be running on different processors, but they will only be running ONE at a time due to GIL.
 - Tasks that spend much of their time waiting for external events are generally good candidates for threading. 
 - Problems that require heavy CPU computation and spend little time waiting for external events might not run faster at all.

In [1]:
import time
import threading

def calc_square(numbers):
    print("calculate square numbers")
    for n in numbers:
        time.sleep(0.1)
        print('square:',n*n)

def calc_cube(numbers):
    print("calculate cube of numbers")
    for n in numbers:
        time.sleep(0.1)
        print('cube:',n*n*n)

arr = [2,3,8,9]

t = time.time()

t1 = threading.Thread(target=calc_square, args=(arr,))
t2 = threading.Thread(target=calc_cube, args=(arr,))

t1.start()
t2.start()

t1.join() # Tells the process to wait until it finishes t1
t2.join() # Tells the process to wait until it finishes t2



print("done in : ",time.time()-t)
print("Hah... I am done with all my work now!")



calculate square numbers
calculate cube of numbers
square: 4
cube: 8
square: 9
cube: 27
square: 64
cube: 512
square: 81
cube: 729
done in :  0.43493127822875977
Hah... I am done with all my work now!


### Daemons
 - It is a thread which will end once the program ends.
 - Once the process will reach end, the daemon threads will auto get killed.


In [ ]:
t1 = threading.Thread(target=thread_function, args=(1,), daemon=True)

### Join 
 - Main process will wait for thread to finish
 - Below it will wait for t1 to get complete.

In [ ]:
t1 = threading.Thread( target = calc_square, args=([1,2],))
t1.start()
t1.join()
print ("all done ...")

### ThreadPoolExecutor
 - Used for starting a pool of threads
 - Below used as context managers
 - Unfortunately, ThreadPoolExecutor will hide that exception. Error program terminates with no output. 

In [ ]:
if __name__ == "__main__":
    with concurrent.futures.ThreadPoolExecutor(max_workers=3) as executor:
        executor.map(calc_square, ([4, 5],))

### Race Condition
 - two or more threads access a shared piece of data or resource.
 - .submit(function, *args, **kwargs)

In [ ]:
database = FakeDatabase()
logging.info("Testing update. Starting value is %d.", database.value)
with concurrent.futures.ThreadPoolExecutor(max_workers=3) as executor:
    for index in range(2):
        executor.submit(database.update, index)
logging.info("Testing update. Ending value is %d.", database.value)

### Basic Synchronization Using Lock: Avoid Race condition
 - A mutex. Mutex comes from MUTual EXclusion, which is exactly what a Lock does.
 - Only one thread at a time can have the Lock.
 - Python’s Lock will also operate as a context manager, so you can use it in a with statement, and it gets released    
   automatically when the with block exits for any reason.

In [7]:
# i will be shared across threads, so multi threaded apps  will print inconsistent results.
i=0; # a global variable
for x in range(100):
    print(i)
    i+=1;

###  Deadlock
 - i.e - Dining Philosophers Problem.
 - Causes
   - An implementation bug where a Lock is not released properly
   - A design issue where a utility function needs to be called by functions that might or might not already have the Lock

In [ ]:
import threading

l = threading.Lock()
print("before first acquire")
l.acquire()
print("before second acquire")
l.acquire()
print("acquired lock twice")
# Deadlock happened as second time it is trying to acquire lock.

before first acquire
before second acquire


### Semaphores
 - A semaphore is simply a variable. 
 - internal counter which is decremented each time acquire() is called and incremented each time release() is called. 
 - makes sense when you want to control access to a resource with limited capacity like a server.

In [ ]:
semaphore = threading.Semaphore()
semaphore.acquire()
# work on a shared resource
...
semaphore.release()

# GIL
 - type of process lock which is used by python whenever it deals with processes.
 - Only one thread can run at a time to avoid reference counter problem.
 - performance of the single-threaded process and the multi-threaded process will be the same.
 - Done to protect memory leak and deadlock b/w threads, use single interpretor lock called GIL.

In [4]:
# The CPython garbage collector uses an efficient memory management technique known as reference counting. 
# Due to this counter, we can count the references and when this count reaches to zero the variable or 
# data object will be released automatically.
import sys
a = []
b = a
sys.getrefcount(a)
# 3
# reference count for the empty list object [] was 3.

3

### Types of multi threading program
- CPU bound - Many operations simultaneosuly
- I/O bound - spend time waiting for Input/Output.

### Diff b/w lock, mutex and semaphore
 - lock: Only allows one thread, not shared with other process
 - Mutex: same as a lock but it can be system wide (shared by multiple processes).
 - Semaphore: Allows x number of threads to enter. Used for limiting cpu, ram usage.

#### Deadlock :

This happens when 2 or more threads are waiting on each other to release the resource for infinite amount of time.
In this the threads are in blocked state and not executing. 
This happens due to buggy code.

e.g - T1 acqired resorce A and then will acquire B
      T2 acqired resorce B and then will acquire A
      Both threads waiting for other resource to get released.

#### Race/Race Condition:

This happens when 2 or more threads run in parallel but end up giving a result which is wrong and not equivalent if all the operations are done in sequential order.
Here all the threads run and execute there operations.

e.g - two people withdrawing some amount from same bank account using ATM
       Total money - Rs 100  
       Person 1 - 10 Rs
       Person 2 - 50 Rs
       
       End result will be Rs 50, but it should be Rs 40.

In Coding we need to avoid both race and deadlock condition.